In [1]:
import pandas as pd

In [2]:
datos = pd.read_excel('./INPUT/MENSAJES/CONSOLIDADO.xlsx', engine = 'openpyxl')

In [3]:
# COMPORTAMIENTO
SIN_COLUMNA_JORNADA = False
ERROR_SI_NO_HAY_JORNADA = True
JORNADA_POR_DEFECTO = 'DÍA'

# VARIABLES DE EXTRACCIÓN DE DATOS
COLUMNA_JORNADA = 'JORNADA'
COLUMNA_CURSO = 'CURSO'
COLUMNA_TEL = 'TELEFONO'
COLUMNA_NOMBRE = 'NOMBRES'
COLUMNA_HORA_INI = 'HORA_INICIO'
COLUMNA_HORA_FIN = 'HORA_FINAL'
COLUMNA_LINK = 'ENLACE_CURSO'
COLUMNA_FECHA = 'FECHA'
COLUMNA_CODIGO = 'CODIGO'
COLUMNA_CEDULA = 'DOCUMENTO'

# Es importante tener en cuenta las banderas {nombre}, {fecha}, {jornada}, {hora_ini}, {hora_fin}, {nombre}
# {link}, {usuario}, {password}

MENSAJE = """MISION TIC PRUEBA DE CARACTERIZACION. Apreciado(a) tripulante {nombre} con código {usuario}, recibe un cordial saludo de parte de la Universidad Industrial de Santander quien te acompañará en la formación de MISIÓN TIC 2022. Para nosotros es muy importante conocerte y para ello hemos creado una prueba de caracterización que nos permitirá identificar tus conocimientos previos en programación e inglés y brindarte una mejor experiencia en los ciclos de formación. Cabe resaltar que esta prueba no descalifica ni incide en la calificacion del proceso formativo, por lo tanto te invitamos a que la desarrolles de manera individual y totalmente a conciencia. Para ingresar a la prueba ingresa al enlace: {link} la cual debera presentar el dia {fecha} en la jornada {jornada} entre las {hora_ini} y {hora_fin} Las credenciales de ingreso -> Usuario {usuario} Contraseña {password}. Es importante que te presentes puntualmente en la fecha y hora programada y accedas con las credenciales proporcionadas anteriormente."""

print ('Longitud del mensaje', len(MENSAJE))

Longitud del mensaje 1013


In [4]:
def calcularCostoAprox(longitudMsg):
    costo = 19
    while (longitudMsg > 0):
        longitudMsg -= 153
        costo += 1.2
    return costo
def normalizar(msg):
    reemplazos = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ñ", "n")
    )
    for a, b in reemplazos:
        msg = msg.replace(a, b).replace(a.upper(), b.upper())
    return msg
def calcularJornada(hora):
    JORNADAS = {
        '8 A.M.': 'MAÑANA',
        '2 P.M.': 'TARDE',
        '6 P.M.': 'NOCHE'
    }
    if hora in JORNADAS:
        return JORNADAS[hora]
    if ERROR_SI_NO_HAY_JORNADA:
        raise Exception('No existe una jornada para la hora: ' + hora)
    return JORNADA_POR_DEFECTO

In [5]:
output = []
serie = []
costoTotal = 0
for index, row in datos.iterrows():    
    msg = normalizar(MENSAJE.format(
        nombre = row[COLUMNA_NOMBRE],
        fecha = row[COLUMNA_FECHA],
        jornada = row[COLUMNA_JORNADA] if not(SIN_COLUMNA_JORNADA) else calcularJornada(row[COLUMNA_HORA_INI]),
        hora_ini = row[COLUMNA_HORA_INI],
        hora_fin = row[COLUMNA_HORA_FIN],
        link = row[COLUMNA_LINK],
        usuario = row[COLUMNA_CODIGO],
        password = row[COLUMNA_CEDULA]
    ))
    costoTotal += calcularCostoAprox(len(msg))
    
    serie.append(msg)
    output.append([
        int(row['TELEFONO']),
        msg
    ])
datos['MENSAJE'] = pd.Series(serie)
print ('Costo aproximado de envío masivo:', costoTotal * 1.1)

Costo aproximado de envío masivo: 147586.99999999837


In [6]:
print(datos['MENSAJE'][0])

MISION TIC PRUEBA DE CARACTERIZACION. Apreciado(a) tripulante STEVEN con codigo 2220001, recibe un cordial saludo de parte de la Universidad Industrial de Santander quien te acompanara en la formacion de MISION TIC 2022. Para nosotros es muy importante conocerte y para ello hemos creado una prueba de caracterizacion que nos permitira identificar tus conocimientos previos en programacion e ingles y brindarte una mejor experiencia en los ciclos de formacion. Cabe resaltar que esta prueba no descalifica ni incide en la calificacion del proceso formativo, por lo tanto te invitamos a que la desarrolles de manera individual y totalmente a conciencia. Para ingresar a la prueba ingresa al enlace: https://lms.uis.edu.co/mintic2022/course/view.php?id=46 la cual debera presentar el dia viernes, 25 de marzo de 2022 en la jornada NOCHE entre las 6 P.M. y 10 P.M. Las credenciales de ingreso -> Usuario 2220001 Contrasena 1013605766. Es importante que te presentes puntualmente en la fecha y hora progr

In [7]:
pd.DataFrame(output).to_excel('./OUTPUT/MENSAJES/FORMATO_SMS.xlsx', index = False, header = False)
datos.to_excel('./OUTPUT/MENSAJES/CONSOLIDADO_SMS.xlsx', index = False)